In [1]:
from pyspark import SparkContext
import csv
from itertools import combinations
from collections import OrderedDict,Counter
import random
from numpy.random import rand

In [2]:
sc = SparkContext.getOrCreate()

input_file = "/Users/liangsiqi/Documents/Dataset/yelp_rec_data/yelp_train.csv"
# minPartition = 5
# raw_data = sc.textFile(input_file,minPartition) #(input_file, minPartition)
# header = raw_data.first()
# clean_data = raw_data.filter(lambda x: x != header).mapPartitions(lambda x: csv.reader(x))

# print(raw_data.count(),clean_data.count())

In [12]:
raws = []
with open(input_file,'r') as f:
    csv_reader = csv.reader(f, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            raws.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

Processed 455855 lines.


In [14]:
clean_data = sc.parallelize(raws)

In [11]:
clean_data.take(3)

[]